# Implementation of an Autoencoder (AE)
#### based on PyTorch tutorial https://pytorch.org/tutorials/beginner/basics/intro.html and https://www.cs.toronto.edu/~lczhang/360/lec/w05/autoencoder.html
## Simple linear AE with one layer into hidden (latent) space 748 -> 512 -> 10 and back 10 -> 512 -> 748

#### Build for MINST datasets

###### ***For sc-RNAseq:*** fix enable cuda/GPU (only needed if CPU is to slow), try different optimiser, check for loss function, trying learning rates, nn.ReLU() function may not be the best, checking for overfitting by plotting loss of model and training data, Batch size has to be optimised: https://arxiv.org/abs/1609.04836 , https://arxiv.org/abs/1703.04933 . Maybe we should try a program like https://opendatascience.com/optimizing-pytorch-performance-batch-size-with-pytorch-profiler/ for that on a later stage for performance (not sure if the data will be to large in the future)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

In [2]:
batch_size = 64

transform = transforms.ToTensor()

mnist_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_test = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_dataloader = DataLoader(dataset=mnist_data,batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(dataset=mnist_data,batch_size=batch_size, shuffle=True)

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"   ### needed if Nvidia GPU is available and wanted to use the GPU
print(f"Using {device} device")

Using cuda device


In [4]:
### Building the neural network structure ####
dim_encoder_decoder = 28 * 28

class linear_AE(nn.Module):
    def __init__(self):
        super(linear_AE, self).__init__()

        self.encoder = nn.Sequential(             # sequential operation of the following code:
            nn.Linear(dim_encoder_decoder, 512),  #### data into the neural network layer (28*28 pixel -> 512 nodes)
            nn.ReLU(),                            #### ReLU(x) = (x)^(+) = max(0,x) 
            nn.Linear(512, 10),                   #### layer into the hidden layer / latent space (512 -> 10 nodes / latent vector)
        )
        
        self.decoder = nn.Sequential(             # sequential operation of the following code:
            nn.Linear(10, 512),                   #### decoding laten layer (10 -> 512 nodes)
            nn.ReLU(),                            #### 
            nn.Linear(512, dim_encoder_decoder),  #### reconstruction of image (512 -> 748)
            nn.Sigmoid()
        )
        
    def forward(self, x):                         # exicute the endcoder and decoder defined in __init__(self)
        endoded = self.encoder(x) 
        decoded = self.decoder(endoded) 
        return decoded
        
model = linear_AE()  #.to(device) #### this is needed for cuda
print(model)

linear_AE(
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=10, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=10, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=784, bias=True)
    (3): Sigmoid()
  )
)


In [5]:
loss_fn = nn.MSELoss ###CrossEntropyLoss()               ### nn.MSELoss (Mean Square Error) for regression tasks

learning_rate = 1e-3                          ### how much the parameter gets updated
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3, weight_decay=1e-5)

In [6]:
num_epochs = 10
outputs = []
torch.manual_seed(42)
for epoch in range(num_epochs):
    for (X, label) in train_dataloader:
        X = X.reshape(-1, 28*28)
        pred = model(X)
        loss = loss_fn(pred, X)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch:{epoch+1}, Loss:{loss.item():.4f}', end="")
    outputs.append((epoch, X, pred))
    
    for (X_test, label) in test_dataloader:
        X_test = X_test.reshape(-1, 28*28)
        pred_test = model(X_test)
        loss_test = loss_fn(pred_test, X_test)
    print(f' ; Train Loss:{loss_test.item():.4f}')

Epoch:1, Loss:0.0261 ; Train Loss:0.0191
Epoch:2, Loss:0.0168 ; Train Loss:0.0186
Epoch:3, Loss:0.0172 ; Train Loss:0.0188
Epoch:4, Loss:0.0174 ; Train Loss:0.0197
Epoch:5, Loss:0.0167 ; Train Loss:0.0172
Epoch:6, Loss:0.0202 ; Train Loss:0.0175
Epoch:7, Loss:0.0185 ; Train Loss:0.0144
Epoch:8, Loss:0.0160 ; Train Loss:0.0161
Epoch:9, Loss:0.0160 ; Train Loss:0.0191
Epoch:10, Loss:0.0171 ; Train Loss:0.0139


In [7]:
torch.save(model, 'model_oneLayer_AE.pth')

In [8]:
model = torch.load('model_oneLayer_AE.pth')

In [9]:
model

linear_AE(
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=10, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=10, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=784, bias=True)
    (3): Sigmoid()
  )
)